In [1]:
import torch 
from torch import nn

import ray
from ray.rllib.agents import ppo
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.annotations import override

#from models import VisualEncoder
from train import *
from wrappers_2 import *

/root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
class VisualEncoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=8, stride=4, padding=0),  
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0), 
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0), 
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(), 
            nn.Conv2d(64, 512, kernel_size=2, stride=1, padding=0),
            nn.ReLU(),
            nn.Flatten(),
        )

    def forward(self, x):
        return self.cnn(x)

In [3]:
class MyModelClass(TorchModelV2, nn.Module):
    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs, model_config, name)
        nn.Module.__init__(self)
        features_dim = 512
        self.encoder = VisualEncoder()
        self.encoder.load_state_dict(
            torch.load("/IGLU-Minecraft/models/AnnaCNN/encoder_weigths.pth", map_location=torch.device('cpu'))
        )
        self.action_head = nn.Linear(features_dim, action_space.n)
        self.value_head = nn.Linear(features_dim, 1)
        self.last_value = None
        
        self.use_cuda = torch.cuda.is_available()
        if self.use_cuda:
            self.encoder.cuda()
            self.action_head.cuda()
            self.value_head.cuda()
        
    @override(TorchModelV2)
    def forward(self, input_dict, state, seq_lens):
        obs = input_dict['obs'].permute(0, 3, 1, 2).float() / 255.0
        if self.use_cuda:
            obs.cuda()
            
        features = self.encoder(obs)
        action = self.action_head(features)
        self.last_value = self.value_head(features).squeeze(1)
        return action, state
    
    @override(TorchModelV2)
    def value_function(self):
        assert self.last_value is not None, "must call forward() first"
        return self.last_value

In [4]:
ModelCatalog.register_custom_model("my_torch_model", MyModelClass)

In [9]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

class RewardWrapper(gym.RewardWrapper):
    def __init__(self, env):
        super().__init__(env)
    
    def reward(self, rew):
        print(rew)
        if rew == 0:
            rew = -0.1
        return rew
    
def env_creator(env_config):
    env = gym.make('IGLUSilentBuilder-v0', max_steps=1000)
    env.update_taskset(TaskSet(preset=['C32']))
    env = PovOnlyWrapper(env)
    env = SelectAndPlace(env)
    env = Discretization(env, flat_action_space('human-level'))
    env = RewardWrapper(env)
    return env

from ray.tune.registry import register_env
register_env("my_env", env_creator)

from ray import tune
from ray.rllib.agents.ppo import PPOTrainer

In [ ]:
from ray.tune.integration.wandb import WandbLogger

analysis = tune.run(PPOTrainer, 
         config={
             "env": "my_env", 
             "framework": "torch",
             "num_gpus": 1,
             "num_workers": 1,
             "sgd_minibatch_size": 256,
             "clip_param": 0.2,
             "entropy_coeff": 0.01,
             "lambda": 0.95,
             "train_batch_size": 1000,
             "model": {
                    # Specify our custom model from above.
                    "custom_model": "my_torch_model",
                    # Extra kwargs to be passed to your model's c'tor.
                    "custom_model_config": {},
              },
             "logger_config": {
                  "wandb": {
                      "project": "IGLU-Minecraft",
                      "name": "PPO C32 pretrained (AnnaCNN) (3 noops after placement and reward shaping)"
                  }
              }

        },
        loggers=[WandbLogger])

Trial name,status,loc
PPO_my_env_5d62f_00000,PENDING,


2021-10-09 19:44:49,979	INFO wandb.py:170 -- Already logged into W&B.
wandb: Currently logged in as: linar (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
(pid=252520) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=252520)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


(pid=252520) 2021-10-09 19:44:53,477	INFO ppo.py:159 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(pid=252520) 2021-10-09 19:44:53,478	INFO trainer.py:728 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=252519) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=252519)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=252520) 2021-10-09 19:44:59,336	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(pid=252519) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=252519)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=252519) /root/miniconda/envs/py37/lib/py

(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=25251

(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=25251

...

(pid=252519) 0
(pid=252519) -2
(pid=252519) 0
(pid=252519) -1
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 1
(pid=252519) 2
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) -2
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 2
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=25

(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) -1
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 2
(pid=252519) 0
(pid=252519) 2
(pid=252519) 2
(pid=252519) 2
(pid=252519) 0
(pid=252519) 0
(pid=252519) -2
(pid=252519) 2
(pid=252

(pid=252519) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=252519)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 2
(pid=252519) 2
(pid=252519) 2
(pid=252519) 2
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 2
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 2
(pid=252519) 2
(pid=252519) 0
(pid=252519) 2
(pid=252519) -2
(pid=252519) 2
(pid=252519) 2
(pid=252519) -2
(pid=252519) 2
(pid=252519) -2
(pid=252519) 2
(pid=252519) 0
(pid=252519) 2
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 2
(pid=252519) 2
(pid=252519) 2
(pid=252519) 2
(pid=252519) 0
(pid=25

(pid=252519) 2
(pid=252519) 0
(pid=252519) 2
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 2
(pid=252519) 2
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) -1
(pid=252519) 0
(pid=252519) -1
(pid=252519) 0
(pid=252519) 2
(pid=252519) 2
(pid=252519) 0
(pid=252519) -2
(pid=252519) 2
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 2
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 2
(pid=252519) -2
(pid=252519) 2
(pid=252519) 2
(pid=252519) 0
(pid=252519) 0
(pid=252519) 2
(pid=252519) 2
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=252519) 0
(pid=2